# Module 04: Advanced Folder Organization

**Difficulty**: ⭐⭐⭐
**Estimated Time**: 45 minutes
**Prerequisites**: [Module 00](00_music_library_manager.ipynb), [Module 01](01_metadata_management.ipynb)

## Learning Objectives
By the end of this notebook, you will be able to:
1. Organize your library using metadata (not filenames)
2. Create Artist/Album folder hierarchies automatically
3. Use folder templates for different organization styles
4. Preview folder structure before applying changes
5. Handle special characters and edge cases safely
6. Reorganize entire library with one command

## Overview
Stop organizing by hand! This module automates folder organization using metadata. Transform a messy flat library into a beautifully structured collection:

**Before:**
```
music/
├── song1.mp3
├── song2.mp3
├── random_folder/
│   ├── track3.mp3
└── ...
```

**After:**
```
music/
├── The Beatles/
│   ├── Abbey Road/
│   │   ├── 01 - Come Together.mp3
│   │   └── 02 - Something.mp3
│   └── Let It Be/
│       └── ...
└── Pink Floyd/
    └── The Wall/
        └── ...
```

### Organization Templates
- **Artist/Album**: `The Beatles/Abbey Road/`
- **Artist/Year - Album**: `The Beatles/1969 - Abbey Road/`
- **Genre/Artist/Album**: `Rock/The Beatles/Abbey Road/`
- **Artist - Album**: `The Beatles - Abbey Road/`
- **Custom**: Define your own!

## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
import shutil
from pathlib import Path
from collections import defaultdict, Counter
from typing import List, Dict, Optional, Tuple
import re
from datetime import datetime

# Metadata handling
from mutagen import File

# For better display
import pandas as pd
from IPython.display import display, HTML
import warnings

# Configuration
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 100)
warnings.filterwarnings('ignore')

print("✓ Libraries imported successfully")

In [ ]:
# Configure paths
MUSIC_LIBRARY_PATH = Path('../../music')
AUDIO_EXTENSIONS = {'.mp3', '.flac', '.wav', '.m4a', '.aac', '.ogg', '.wma', '.opus'}

# Folder organization templates
FOLDER_TEMPLATES = {
    'artist_album': '{artist}/{album}',
    'artist_year_album': '{artist}/{year} - {album}',
    'genre_artist_album': '{genre}/{artist}/{album}',
    'year_artist_album': '{year}/{artist} - {album}',
    'artist_dash_album': '{artist} - {album}',
    'flat_artist_album': '{artist} - {album}',  # No subfolders
}

# Characters to remove/replace in folder names
UNSAFE_CHARS = r'[<>:"/\\|?*]'

if not MUSIC_LIBRARY_PATH.exists():
    MUSIC_LIBRARY_PATH.mkdir(parents=True, exist_ok=True)
    print(f"Created: {MUSIC_LIBRARY_PATH.absolute()}")
else:
    print(f"✓ Music library path: {MUSIC_LIBRARY_PATH.absolute()}")

## 2. Helper Functions for Safe Folder Names

In [ ]:
def sanitize_folder_name(name: str) -> str:
    """
    Make a string safe for use as a folder/file name.
    Removes or replaces unsafe characters.
    
    Args:
        name: Original name
    
    Returns:
        Sanitized name safe for filesystems
    """
    if not name:
        return "Unknown"
    
    # Remove unsafe characters
    sanitized = re.sub(UNSAFE_CHARS, '', str(name))
    
    # Replace multiple spaces with single space
    sanitized = re.sub(r'\s+', ' ', sanitized)
    
    # Remove leading/trailing spaces and dots
    sanitized = sanitized.strip('. ')
    
    # If empty after sanitization, return default
    if not sanitized:
        return "Unknown"
    
    # Limit length (most filesystems support 255 chars)
    if len(sanitized) > 200:
        sanitized = sanitized[:200].strip()
    
    return sanitized


def extract_year_from_date(date_str: Optional[str]) -> Optional[str]:
    """
    Extract year from various date formats.
    Handles: "2020", "2020-05-15", "05/15/2020", etc.
    """
    if not date_str:
        return None
    
    # Try to find 4-digit year
    match = re.search(r'\b(19|20)\d{2}\b', str(date_str))
    if match:
        return match.group(0)
    
    return None


def format_track_number(track_num: Optional[str]) -> str:
    """
    Format track number with leading zeros.
    Handles: "1", "01", "1/12", etc.
    """
    if not track_num:
        return ""
    
    # Extract just the track number (ignore total tracks)
    track_str = str(track_num).split('/')[0].strip()
    
    # Try to convert to int and format
    try:
        track_int = int(track_str)
        return f"{track_int:02d}"  # Two digits with leading zero
    except ValueError:
        return track_str

print("✓ Helper functions loaded")

## 3. Metadata Extraction for Organization

In [ ]:
def extract_organization_metadata(file_path: Path) -> Dict:
    """
    Extract metadata needed for folder organization.
    
    Returns:
        Dictionary with artist, album, title, year, genre, tracknumber
    """
    metadata = {
        'artist': None,
        'album': None,
        'title': None,
        'year': None,
        'genre': None,
        'tracknumber': None
    }
    
    try:
        audio = File(file_path, easy=True)
        
        if audio is None:
            return metadata
        
        # Extract tags
        for tag in ['artist', 'album', 'title', 'genre', 'tracknumber']:
            if tag in audio and audio[tag]:
                value = audio[tag]
                if isinstance(value, list):
                    metadata[tag] = str(value[0]) if value else None
                else:
                    metadata[tag] = str(value)
        
        # Extract year from date tag
        if 'date' in audio and audio['date']:
            date_value = audio['date'][0] if isinstance(audio['date'], list) else audio['date']
            metadata['year'] = extract_year_from_date(str(date_value))
    
    except Exception:
        pass
    
    return metadata


def scan_for_organization(library_path: Path) -> List[Dict]:
    """
    Scan library and extract organization metadata.
    """
    music_files = []
    
    print(f"Scanning for organization: {library_path}")
    
    file_count = 0
    for root, dirs, files in os.walk(library_path):
        for file in files:
            file_path = Path(root) / file
            
            if file_path.suffix.lower() in AUDIO_EXTENSIONS:
                file_count += 1
                
                if file_count % 50 == 0:
                    print(f"  Scanned {file_count} files...")
                
                metadata = extract_organization_metadata(file_path)
                
                file_info = {
                    'filename': file,
                    'path': str(file_path),
                    'current_folder': str(file_path.parent.relative_to(library_path)),
                    'artist': metadata.get('artist'),
                    'album': metadata.get('album'),
                    'title': metadata.get('title'),
                    'year': metadata.get('year'),
                    'genre': metadata.get('genre'),
                    'tracknumber': metadata.get('tracknumber')
                }
                
                music_files.append(file_info)
    
    print(f"✓ Scanned {file_count} files")
    return music_files

print("✓ Metadata extraction functions loaded")

## 4. Folder Path Generation

In [ ]:
def generate_folder_path(file_info: Dict, template: str) -> Optional[str]:
    """
    Generate folder path based on metadata and template.
    
    Args:
        file_info: Dictionary with file metadata
        template: Folder structure template (e.g., '{artist}/{album}')
    
    Returns:
        Folder path string, or None if required metadata is missing
    """
    # Prepare metadata for template
    template_data = {
        'artist': sanitize_folder_name(file_info.get('artist')) if file_info.get('artist') else None,
        'album': sanitize_folder_name(file_info.get('album')) if file_info.get('album') else None,
        'title': sanitize_folder_name(file_info.get('title')) if file_info.get('title') else None,
        'year': file_info.get('year') or 'Unknown Year',
        'genre': sanitize_folder_name(file_info.get('genre')) if file_info.get('genre') else 'Unknown Genre'
    }
    
    # Check if required fields are present
    required_fields = re.findall(r'\{(\w+)\}', template)
    
    for field in required_fields:
        if field in ['artist', 'album', 'title']:  # Critical fields
            if not template_data.get(field) or template_data[field] == 'Unknown':
                return None  # Missing critical metadata
    
    try:
        # Format template
        folder_path = template.format(**template_data)
        return folder_path
    except KeyError:
        return None


def generate_filename_with_track(file_info: Dict) -> str:
    """
    Generate filename with track number prefix if available.
    
    Returns:
        Formatted filename (e.g., "01 - Title.mp3" or "Title.mp3")
    """
    original_filename = file_info['filename']
    track_num = file_info.get('tracknumber')
    title = file_info.get('title')
    extension = Path(original_filename).suffix
    
    # If we have both track number and title
    if track_num and title:
        formatted_track = format_track_number(track_num)
        safe_title = sanitize_folder_name(title)
        return f"{formatted_track} - {safe_title}{extension}"
    
    # If we have title only
    elif title:
        safe_title = sanitize_folder_name(title)
        return f"{safe_title}{extension}"
    
    # Keep original filename
    else:
        return original_filename

print("✓ Folder path generation functions loaded")

## 5. Organization Planning and Preview

In [ ]:
def plan_organization(music_files: List[Dict], 
                      template: str,
                      rename_files: bool = True) -> List[Dict]:
    """
    Plan organization changes without moving files.
    
    Args:
        music_files: List of file information dictionaries
        template: Organization template to use
        rename_files: Whether to rename files with track numbers
    
    Returns:
        List of planned moves with old and new paths
    """
    plan = []
    
    for file_info in music_files:
        # Generate new folder path
        new_folder = generate_folder_path(file_info, template)
        
        if new_folder:
            # Generate new filename if requested
            if rename_files:
                new_filename = generate_filename_with_track(file_info)
            else:
                new_filename = file_info['filename']
            
            # Build new full path
            new_relative_path = str(Path(new_folder) / new_filename)
            
            plan.append({
                'current_path': file_info['path'],
                'current_filename': file_info['filename'],
                'current_folder': file_info['current_folder'],
                'new_folder': new_folder,
                'new_filename': new_filename,
                'new_relative_path': new_relative_path,
                'needs_move': file_info['current_folder'] != new_folder,
                'needs_rename': file_info['filename'] != new_filename,
                'artist': file_info.get('artist'),
                'album': file_info.get('album')
            })
        else:
            # Missing metadata - can't organize
            plan.append({
                'current_path': file_info['path'],
                'current_filename': file_info['filename'],
                'current_folder': file_info['current_folder'],
                'new_folder': None,
                'new_filename': file_info['filename'],
                'new_relative_path': None,
                'needs_move': False,
                'needs_rename': False,
                'status': 'Missing metadata',
                'artist': file_info.get('artist'),
                'album': file_info.get('album')
            })
    
    return plan


def preview_folder_structure(plan: List[Dict], max_folders: int = 20) -> str:
    """
    Generate a preview of the folder structure.
    
    Returns:
        String with tree-like structure preview
    """
    # Collect unique folders
    folders = defaultdict(int)
    for item in plan:
        if item.get('new_folder'):
            folders[item['new_folder']] += 1
    
    # Sort by path
    sorted_folders = sorted(folders.items())
    
    # Build preview
    lines = []
    lines.append("Folder Structure Preview:")
    lines.append("=" * 60)
    lines.append("")
    
    shown = 0
    current_artist = None
    
    for folder_path, count in sorted_folders:
        if shown >= max_folders:
            remaining = len(sorted_folders) - shown
            lines.append(f"... and {remaining} more folders")
            break
        
        # Parse folder path
        parts = Path(folder_path).parts
        
        if len(parts) == 2:  # Artist/Album
            artist, album = parts
            if artist != current_artist:
                lines.append(f"{artist}/")
                current_artist = artist
            lines.append(f"  └── {album}/ ({count} files)")
        else:
            lines.append(f"{folder_path}/ ({count} files)")
        
        shown += 1
    
    return "\n".join(lines)

print("✓ Organization planning functions loaded")

## 6. Organization Execution

In [ ]:
def execute_organization(library_path: Path, 
                        plan: List[Dict],
                        dry_run: bool = True) -> List[Dict]:
    """
    Execute the organization plan.
    
    Args:
        library_path: Root music library path
        plan: Organization plan from plan_organization()
        dry_run: If True, only simulate (don't actually move)
    
    Returns:
        List of actions taken
    """
    actions = []
    
    for item in plan:
        if not item.get('needs_move') and not item.get('needs_rename'):
            continue  # Already in correct location
        
        if not item.get('new_relative_path'):
            continue  # Missing metadata, can't organize
        
        current_path = Path(item['current_path'])
        new_path = library_path / item['new_relative_path']
        
        action = {
            'file': item['current_filename'],
            'from': item['current_folder'],
            'to': item['new_folder'],
            'new_filename': item['new_filename'],
            'status': 'Would move' if dry_run else 'Moved'
        }
        
        if not dry_run:
            try:
                # Create target directory
                new_path.parent.mkdir(parents=True, exist_ok=True)
                
                # Check if target exists
                if new_path.exists():
                    action['status'] = 'Skipped (target exists)'
                else:
                    # Move file
                    shutil.move(str(current_path), str(new_path))
                    action['status'] = 'Moved'
            
            except Exception as e:
                action['status'] = f'Failed: {str(e)}'
        
        actions.append(action)
    
    return actions


def cleanup_empty_folders(library_path: Path, dry_run: bool = True) -> List[str]:
    """
    Remove empty folders after reorganization.
    
    Returns:
        List of removed folder paths
    """
    removed = []
    
    # Walk bottom-up to remove empty folders
    for root, dirs, files in os.walk(library_path, topdown=False):
        root_path = Path(root)
        
        # Skip the library root
        if root_path == library_path:
            continue
        
        # Check if folder is empty
        if not any(root_path.iterdir()):
            if not dry_run:
                root_path.rmdir()
            removed.append(str(root_path.relative_to(library_path)))
    
    return removed

print("✓ Organization execution functions loaded")

---
## 7. Usage Examples

### 7.1 Scan Library

In [ ]:
# Scan library for organization
print("Scanning library...\n")
all_music_files = scan_for_organization(MUSIC_LIBRARY_PATH)

print(f"\nFound {len(all_music_files)} files")

if all_music_files:
    # Show sample
    sample = pd.DataFrame(all_music_files[:10])
    display(sample[['filename', 'artist', 'album', 'title', 'year']])
else:
    print("No music files found.")

### 7.2 View Available Templates

In [ ]:
# Show available organization templates
print("Available Organization Templates\n" + "=" * 60)
print()

for name, template in FOLDER_TEMPLATES.items():
    print(f"{name}:")
    print(f"  Template: {template}")
    print(f"  Example: {template.format(artist='The Beatles', album='Abbey Road', year='1969', genre='Rock')}")
    print()

### 7.3 Preview Organization (Artist/Album)

In [ ]:
# Plan organization using Artist/Album template
if all_music_files:
    print("Planning organization with Artist/Album structure...\n")
    
    # Use the artist_album template
    template = FOLDER_TEMPLATES['artist_album']
    organization_plan = plan_organization(
        all_music_files, 
        template,
        rename_files=True  # Also rename files with track numbers
    )
    
    # Count what needs to be done
    needs_move = sum(1 for p in organization_plan if p.get('needs_move'))
    needs_rename = sum(1 for p in organization_plan if p.get('needs_rename'))
    missing_metadata = sum(1 for p in organization_plan if not p.get('new_folder'))
    
    print(f"Organization Summary:")
    print(f"  Total files: {len(organization_plan)}")
    print(f"  Files to move: {needs_move}")
    print(f"  Files to rename: {needs_rename}")
    print(f"  Files with missing metadata: {missing_metadata}")
    print()
    
    # Show folder structure preview
    if needs_move > 0:
        print(preview_folder_structure(organization_plan, max_folders=15))
else:
    print("No files to organize.")

### 7.4 Show Sample File Moves

In [ ]:
# Show sample of planned moves
if all_music_files and organization_plan:
    # Filter to only files that need changes
    changes = [p for p in organization_plan if p.get('needs_move') or p.get('needs_rename')]
    
    if changes:
        print(f"Sample of Planned Changes (showing first 20):")
        print("=" * 60 + "\n")
        
        df = pd.DataFrame(changes[:20])
        display(df[['current_filename', 'current_folder', 'new_folder', 'new_filename']])
    else:
        print("✓ All files are already organized correctly!")
else:
    print("No changes to preview.")

### 7.5 Try Different Template (Year/Artist/Album)

In [ ]:
# Preview with different template
if all_music_files:
    template = FOLDER_TEMPLATES['artist_year_album']
    print(f"Preview with template: {template}\n")
    
    alt_plan = plan_organization(all_music_files, template, rename_files=True)
    
    print(preview_folder_structure(alt_plan, max_folders=15))
else:
    print("No files to preview.")

### 7.6 Find Files with Missing Metadata

In [ ]:
# Show files that can't be organized due to missing metadata
if all_music_files and organization_plan:
    missing_metadata = [p for p in organization_plan if not p.get('new_folder')]
    
    if missing_metadata:
        print(f"⚠️ {len(missing_metadata)} files cannot be organized (missing metadata):\n")
        df = pd.DataFrame(missing_metadata)
        display(df[['current_filename', 'artist', 'album']].head(20))
        
        print("\n💡 Tip: Use Module 01 to add metadata to these files first")
    else:
        print("✓ All files have sufficient metadata for organization!")
else:
    print("No files to check.")

### 7.7 Execute Organization (DRY RUN)

In [ ]:
# Dry run: Simulate organization
if all_music_files and organization_plan:
    print("Simulating organization (dry run)...\n")
    
    actions = execute_organization(
        MUSIC_LIBRARY_PATH,
        organization_plan,
        dry_run=True
    )
    
    if actions:
        print(f"Would process {len(actions)} files:\n")
        df = pd.DataFrame(actions[:20])
        display(df[['file', 'from', 'to', 'status']])
        
        print(f"\n💡 Showing first 20 of {len(actions)} actions")
        print("\n💡 To actually organize, run with dry_run=False:")
        print("   actions = execute_organization(MUSIC_LIBRARY_PATH, organization_plan, dry_run=False)")
    else:
        print("✓ No changes needed!")
else:
    print("No files to organize.")

### 7.8 Execute Organization (ACTUAL)

In [ ]:
# ACTUAL EXECUTION - Uncomment to use
# WARNING: This will move files in your library!

# if all_music_files and organization_plan:
#     print("🎵 Organizing library (ACTUAL EXECUTION)...\n")
#     print("⚠️ Warning: This will move and rename files!\n")
#     
#     # Execute organization
#     actions = execute_organization(
#         MUSIC_LIBRARY_PATH,
#         organization_plan,
#         dry_run=False  # ACTUALLY MOVE FILES
#     )
#     
#     # Show results
#     success = sum(1 for a in actions if a['status'] == 'Moved')
#     skipped = sum(1 for a in actions if 'Skipped' in a['status'])
#     failed = sum(1 for a in actions if 'Failed' in a['status'])
#     
#     print(f"\n✓ Organization complete!")
#     print(f"  Moved: {success}")
#     print(f"  Skipped: {skipped}")
#     print(f"  Failed: {failed}")
#     
#     # Clean up empty folders
#     print("\nCleaning up empty folders...")
#     removed = cleanup_empty_folders(MUSIC_LIBRARY_PATH, dry_run=False)
#     print(f"  Removed {len(removed)} empty folders")

print("💡 Uncomment the code above to actually organize your library")
print("💡 Always run dry_run=True first to preview changes!")

### 7.9 Custom Template Example

In [ ]:
# Example: Create custom template
if all_music_files:
    # Custom template: Genre/Year/Artist - Album
    custom_template = "{genre}/{year}/{artist} - {album}"
    
    print(f"Custom template: {custom_template}\n")
    
    custom_plan = plan_organization(all_music_files, custom_template, rename_files=False)
    
    print(preview_folder_structure(custom_plan, max_folders=10))
    
    print("\n💡 You can create any template using these variables:")
    print("   {artist}, {album}, {title}, {year}, {genre}")
else:
    print("No files to preview.")

## 8. Summary

### What We've Learned

In this module, we've implemented powerful metadata-based organization:

1. **Template-Based Organization**: Use predefined or custom folder structures
2. **Safe File Handling**: Sanitize names, handle special characters
3. **Smart Renaming**: Add track numbers to filenames automatically
4. **Preview Before Execution**: See exactly what will happen
5. **Metadata Validation**: Identify files missing critical tags
6. **Cleanup**: Remove empty folders after reorganization

### Available Templates

- **artist_album**: `Artist/Album/` - Simple, clean
- **artist_year_album**: `Artist/Year - Album/` - Good for chronological browsing
- **genre_artist_album**: `Genre/Artist/Album/` - Organize by music type
- **year_artist_album**: `Year/Artist - Album/` - Timeline organization
- **artist_dash_album**: `Artist - Album/` - Flat structure alternative

### Key Functions

**Planning:**
- `plan_organization()` - Create organization plan
- `preview_folder_structure()` - Visual preview of result
- `generate_folder_path()` - Build folder path from template

**Execution:**
- `execute_organization()` - Move files to new structure
- `cleanup_empty_folders()` - Remove empty directories

**Safety:**
- `sanitize_folder_name()` - Make names filesystem-safe
- Always use `dry_run=True` first!

### Best Practices

1. **Always Preview First**: Use dry_run=True to see changes
2. **Fix Metadata**: Ensure files have artist/album tags before organizing
3. **Backup**: Keep backups before major reorganization
4. **Start Small**: Test on a subset before entire library
5. **Consistent Template**: Stick to one template for your whole library
6. **Clean Up**: Run cleanup_empty_folders() after reorganization

### Common Workflows

**Full Library Reorganization:**
```python
# 1. Scan
files = scan_for_organization(MUSIC_LIBRARY_PATH)

# 2. Choose template and plan
plan = plan_organization(files, FOLDER_TEMPLATES['artist_album'])

# 3. Preview
print(preview_folder_structure(plan))

# 4. Dry run
actions = execute_organization(MUSIC_LIBRARY_PATH, plan, dry_run=True)

# 5. Execute
actions = execute_organization(MUSIC_LIBRARY_PATH, plan, dry_run=False)

# 6. Clean up
cleanup_empty_folders(MUSIC_LIBRARY_PATH, dry_run=False)
```

### Troubleshooting

**Issue**: Files not moving
- Check if files have required metadata (artist, album)
- Use Module 01 to add missing metadata

**Issue**: "Target exists" errors
- You have duplicate files in different locations
- Review duplicates with Module 02

**Issue**: Unsafe characters in folder names
- `sanitize_folder_name()` automatically handles this
- Special chars are removed/replaced

### Next Steps

Now that your library is organized:
- **Module 06: Visualizations** - Analyze your organized collection
- **Module 07: Playlists** - Create playlists by folder structure
- **Module 08: Validation** - Verify organization quality

### Additional Resources

- [File naming best practices](https://en.wikipedia.org/wiki/Filename)
- [Music library organization strategies](https://www.reddit.com/r/DataHoarder/wiki/music)
- [Metadata standards](https://musicbrainz.org/doc/Style)